In [1]:
import numpy as np

Avoid nececssary array copies

In [3]:
a = np.ones(10)
a

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [4]:
def id(x):
    # This function returns the memory
    # block address of an array.
    return x.__array_interface__['data'][0]

In [7]:
aid = id(a)
aid

60040128L

In [8]:
# if we copy the array, its copy is stored in a dirrenet location in the memory
b = a.copy();
id(b) == aid

False

Inplace and Implicit copy operations

In [9]:
# an inplacec operation
a *= 2
print a
id(a) == aid

[ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]


True

In [13]:
# c is newly created
c = a * 2
print id(c) == aid
# hm, really?
print id(c)

False
60036768


In [21]:
# Implicit copy operations are significantly slower
%alias_magic t timeit
%t a = np.zeros(10000000)
a *= 2

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.
1000 loops, best of 3: 939 µs per loop


In [22]:
%t a = np.zeros(10000000)
c = a * 2

1000 loops, best of 3: 861 µs per loop


In [24]:
# respahing an array does not involve a copy
a = np.zeros((10, 10)); aid = id(a)
print "a's id " + str(aid)
b = a.reshape((1, -1)); 
print id(b) == aid
print id(b)

a's id 70748560
True


In [25]:
# transposing does involve a copy
# thus it is slower than just reshaping an array
c = a.T.reshape((1, -1)); id(c) == aid

False

Flatten and Ravel

In [28]:
# flatten returns a vector of the numpy array, makes a copy
d = a.flatten(); id(d) == aid

False

In [29]:
# ravel also returns a vector of the numpy array, does NOT make a copy, thus is faster
e = a.ravel(); id(e) == aid

True

In [30]:
%t a.flatten()

The slowest run took 11.33 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 3.25 µs per loop


In [31]:
%timeit a.ravel()# use this

The slowest run took 16.85 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 914 ns per loop


Broadcasting Rules

In [32]:
n = 1000
a = np.arange(n)
# creates an axis of len 1
ac = a[:, np.newaxis]
ar = a[np.newaxis, :]

In [34]:
# a row vector
ac.shape

(1000L, 1L)

In [35]:
# a column vector
ar.shape

(1L, 1000L)

In [36]:
# multiply vectors
%t np.tile(ac, (1, n)) * np.tile(ar, (n, 1))

10 loops, best of 3: 38.4 ms per loop


In [38]:
# multiply vectors - faster
%t ar * ac

100 loops, best of 3: 10.4 ms per loop


Efficient Selections in Arrays

In [40]:
# let's create an array with a large number of rows
n, d = 100000, 100
a = np.random.random_sample((n, d));
aid = id(a)
a.shape

(100000L, 100L)

In [41]:
# selection 1
b1 = a[::10]
# selection 2
b2 = a[np.arange(0, n, 10)]
# equal? - yes
np.array_equal(b1, b2)

True

In [42]:
# inplace operation, implicit copy
id(b1) == aid, id(b2) == aid

(True, False)

In [43]:
%t a[::10] # use this

The slowest run took 15.77 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 868 ns per loop


In [44]:
%t a[np.arange(0, n, 10)]

100 loops, best of 3: 12.4 ms per loop


Indexing

In [62]:
# mask for boolean
i = np.random.random_sample(n) < .5
print i.shape
print i[:10]
b1 = a[i]
print b1.shape
b2 = np.compress(i, a, axis=0)
print b2.shape
np.array_equal(b1, b2)

(100000L,)
[False False  True  True False  True  True False False False]
(50180L, 100L)
(50180L, 100L)


True

In [54]:
%t a[i] # use this

100 loops, best of 3: 12.3 ms per loop


In [64]:
%t np.compress(i, a, axis=0)

10 loops, best of 3: 62.2 ms per loop


Mask v.s Where

In [73]:
# with a mask
mask = a < .5
b1 = a[mask]
print b1.shape

# with where 
b2 = a[np.where(a < .5)]
print b2.shape
np.array_equal(b1, b2)

(4999416L,)
(4999416L,)


True

In [74]:
%t a[mask] # use this

1 loops, best of 3: 286 ms per loop


In [75]:
%t a[np.where(a < .5)]

1 loops, best of 3: 656 ms per loop


Unfunc (Universal Function) at

In [89]:
a = np.array([1, 2, 3, 4])
# at(matrix, indices to be modified, value to be changed)
# increment item 0 once, and increment item 2 trice, do not change item 1 and 3
np.add.at(a, [0, 2, 2, 2], 1)
a

array([2, 2, 6, 4])

In [90]:
# partial addition 1
a = np.array([1, 2, 3, 4])
# at(matrix, indices to be modified, value to be changed)
# increment item 0 and 1 once, and increment item 2 twice, do not change last item
np.add.at(a, [0, 1, 2, 2], 1)
a

array([2, 3, 5, 4])

In [93]:
# partial addition 2
a = np.array([1, 2, 3, 4])
def addition(x):
    x[0] += 1
    x[1] += 1
    x[2] += 2
addition(a)
a

array([2, 3, 5, 4])

In [96]:
%t np.add.at(a, [0, 1, 2, 2], 1)

The slowest run took 6.19 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 20.4 µs per loop


In [97]:
%t addition(a)# use this

The slowest run took 7.40 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 3.93 µs per loop


Unfunc negative

In [86]:
a = np.array([1, 2, 3, 4])
np.negative.at(a, [0, 1])
a

array([-1, -2,  3,  4])

Adding Columns to Matrices

In [122]:
# example of r_
a = np.array([[0, 1, 2], [3, 4, 5]])
b = np.array([[10],[20]])
print a
print "\n"
# concatenate along last axis
print np.r_['-1', a, b]
print "\n"

[[0 1 2]
 [3 4 5]]


[[ 0  1  2 10]
 [ 3  4  5 20]]




In [128]:
%t np.append(a, b, axis=1) # use this one

The slowest run took 6.97 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 8.22 µs per loop


In [129]:
%t np.r_['-1', a, b]

10000 loops, best of 3: 65.1 µs per loop


In [130]:
%t np.hstack((a,b))

The slowest run took 4.14 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 16.2 µs per loop
